In [22]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import datetime as dt

In [3]:
df = pd.read_csv('children-growth.csv')

In [4]:
df.head()

,PERSON,DATE,WEIGHT,HEIGHT,HEAD
0,Ariel,2014.10.17,14.0,92.2,NaN
1,Ariel,2014.10.4,13.5,90.8,NaN
2,Ariel,2014.6.26,12.8,90.7,NaN
3,Ariel,2014.8.22,13.8,92.0,NaN
4,Ted,2014.9.23,11.3,77.3,NaN


In [5]:
df.shape

(77, 5)

In [13]:
df.DATE = pd.to_datetime(df.DATE)

In [14]:
df.dtypes

PERSON            object
DATE      datetime64[ns]
WEIGHT           float64
HEIGHT           float64
HEAD             float64
dtype: object

In [6]:
df.describe()

,WEIGHT,HEIGHT,HEAD
count,77.000000,77.000000,43.000000
mean,20.315584,106.763636,50.253488
std,6.372785,14.687570,1.717466
min,11.300000,77.300000,46.000000
25%,16.000000,96.200000,49.250000
50%,19.200000,105.500000,50.000000
75%,22.200000,116.000000,52.000000
max,37.100000,136.700000,53.500000


In [7]:
df.PERSON.unique()

array(['Ariel', 'Ted', 'Jua'], dtype=object)

In [15]:
birth = {'Ariel':'2012-01-28', 'Ted':'2013-09-28', 'Jua':'2014-11-10'}

In [16]:
df.head()

,PERSON,DATE,WEIGHT,HEIGHT,HEAD
0,Ariel,2014-10-17,14.0,92.2,NaN
1,Ariel,2014-10-04,13.5,90.8,NaN
2,Ariel,2014-06-26,12.8,90.7,NaN
3,Ariel,2014-08-22,13.8,92.0,NaN
4,Ted,2014-09-23,11.3,77.3,NaN


In [27]:
for date in birth:
    print(date)
#     dt.datetime.strptme(birth, '%Y-%m-%d').date()

Ariel
Ted
Jua
